In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

In [2]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated.csv")

In [3]:
patient_id = total["patient_id"].to_list()
patient_id = patient_id[:10]

In [4]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches.csv")

In [ ]:
# ONLY RUN ONCE #

# vertifying that all match records can be found
for i in range(len(patient_id)):
    specific_matches = matches[matches["PatientID"] == patient_id[i]]
    if (len(specific_matches) <= 0):
        print("NOT FOUND: ", patient_id[i])
    #specific_matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_{}.csv".format(i+1))

In [160]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_1.csv")

In [6]:
unique_ids = matches["PatientID"].unique().tolist()
print(len(unique_ids))

2890


In [7]:
def get_match(query, choices, limit = 1):
    results = process.extract(query, choices, limit = limit)
    return results

In [9]:
# interesting match cases
# patient_2 in specific study row 6, cream that has mci inside it

In [111]:
subset_loc = p1[p1["PatientID"] == 'Z6352398']['regex_location'].to_list()
subset_loc_clone = p1[p1["PatientID"] == 'Z6352398']['regex_location'].to_list()
#print(subset_loc)
encounters = p1[p1["PatientID"] == 'Z6352398']['PatientEncounterID'].unique().tolist()

#print(len(subset_loc))
#print(subset_loc)
#print(subset_loc[0])
for i in range(len(subset_loc)):
    subset_loc[i] = subset_loc[i][1:len(subset_loc[i]) - 1]
    subset_loc[i] = eval(subset_loc[i])
    #subset_loc[i] = tuple(map(int, subset_loc[i].split(", ")))
    subset_loc_clone[i] = subset_loc_clone[i][1:len(subset_loc_clone[i]) - 1]
    subset_loc_clone[i] = eval(subset_loc_clone[i])
    
subset_loc = pd.Series(subset_loc)
subset_loc = subset_loc.unique().tolist()

subset_loc_clone = pd.Series(subset_loc_clone)
subset_loc_clone = subset_loc_clone.unique().tolist()

print("CLONE: ", subset_loc_clone)
print(subset_loc)
print("\n", subset_loc[0][0])
print(len(subset_loc))
if (len(subset_loc) != len(encounters)):
    print("Houston, we got a problem", i, len(subset_loc), len(encounters))

total_merged = []  

for i in range(len(subset_loc)):
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(subset_loc[i]) == 2):
        start = subset_loc[i][0]
        end = subset_loc[i][1]
        length = len(p1[(p1["regex_location"] == "[" +str(subset_loc[i]) + "]") & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[0])
        if (start < 100):
            start = 100
        elif (end > length - 100):
             end = length - 100
        subset_loc[i] = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(subset_loc[i])):
            start = subset_loc[i][j][0]
            end = subset_loc[i][j][1]
            loc_list.append((start, end))
        length = len(p1[(p1["regex_location"] == str(loc_list)) & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[1])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        subset_loc[i] = loc_list
    
    subset_loc[i].sort(key = lambda x : x[0])
    
    print("\nSUBSET_LOC", subset_loc[i])
    
    for interval in subset_loc[i]:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
    total_merged.append(merged)
    
total_merged
#     print("start", start)
#     print("end", end)
    #print("matches, ", p1[(p1["regex_location"] == "[" +str(subset_loc[j]) + "]") & (p1["PatientEncounterID"] == int(encounters[j]))]['NoteTXT'].values)
    #print("length, ", length) 
    

CLONE:  [(5850, 5858), (3643, 3649), (6049, 6057), (3810, 3816), (5554, 5562), (5634, 5642), (5492, 5500), (5891, 5899), (5790, 5798), (6170, 6178), (6128, 6136), (6061, 6069), (6355, 6363), (3947, 3950), (8741, 8749), ((1048, 1052), (4878, 4887), (5104, 5110), (5114, 5123), (5227, 5231)), ((5467, 5476), (5693, 5699), (5703, 5712)), (1984, 1990), (859, 862), ((281, 290), (579, 588), (2148, 2154), (2163, 2169), (6284, 6290), (6684, 6693), (6718, 6721), (7550, 7559), (7816, 7820), (7828, 7832), (9964, 9972), (11292, 11301), (11383, 11403), (11514, 11523), (11544, 11548), (12218, 12238), (12383, 12392), (12487, 12507), (12509, 12512), (12519, 12527)), (313, 319), ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862)), (1473, 1476)]
[(5850, 5858), (3643, 3649), (6049, 6057), (3810, 3816), (5554, 5562), (5634, 5642), (5492, 5500), (5891, 5899), (5790, 5798), (6170, 6178), (6128, 6136), (6061, 6069), (6355, 6363), (3947, 3950), (8741, 8749), ((1048, 1052), (4878, 4887), (5104, 5110), (5114

[[(5750, 5958)],
 [(3543, 3749)],
 [(5949, 6157)],
 [(3710, 3916)],
 [(5454, 5662)],
 [(5534, 5742)],
 [(5392, 5600)],
 [(5791, 5999)],
 [(5690, 5898)],
 [(6070, 6278)],
 [(6028, 6236)],
 [(5961, 6169)],
 [(6255, 6463)],
 [(3847, 4050)],
 [(8641, 8849)],
 [(948, 1152), (4778, 4987), (5004, 5331)],
 [(5367, 5576), (5593, 5812)],
 [(1884, 2090)],
 [(759, 962)],
 [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)],
 [(213, 419)],
 [(7409, 7618), (7635, 7962)],
 [(1373, 1576)]]

In [112]:
print(subset_loc_clone)

[(5850, 5858), (3643, 3649), (6049, 6057), (3810, 3816), (5554, 5562), (5634, 5642), (5492, 5500), (5891, 5899), (5790, 5798), (6170, 6178), (6128, 6136), (6061, 6069), (6355, 6363), (3947, 3950), (8741, 8749), ((1048, 1052), (4878, 4887), (5104, 5110), (5114, 5123), (5227, 5231)), ((5467, 5476), (5693, 5699), (5703, 5712)), (1984, 1990), (859, 862), ((281, 290), (579, 588), (2148, 2154), (2163, 2169), (6284, 6290), (6684, 6693), (6718, 6721), (7550, 7559), (7816, 7820), (7828, 7832), (9964, 9972), (11292, 11301), (11383, 11403), (11514, 11523), (11544, 11548), (12218, 12238), (12383, 12392), (12487, 12507), (12509, 12512), (12519, 12527)), (313, 319), ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862)), (1473, 1476)]


In [130]:
total_note_txt = []
print(len(total_merged))
for i in range(len(total_merged)):
    curr = ""
    if (len(total_merged[i]) == 1):
        note = p1[(p1["regex_location"] == "[" +str(subset_loc_clone[i]) + "]") & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[0]
        #print(note)
        #print(total_merged[i][0][0], total_merged[i][0][1])
        curr = note[total_merged[i][0][0]:total_merged[i][0][1]]
    else:
        for j in range(len())
    total_note_txt.append(curr)
    
total_note_txt

23


['ncer. She passed secondary to this. He does not know his father. He thinks his father passed from a cerebral hemorrage. His niece had breast cancer diagnosed in her 50s. He had one sister who passed from a fi',
 'a. Skin: No rashes or lesions. Neurological: Awake, alert and oriented x 3. Attention, language and memory are intact. Cranial nerve II-XII are grossly intact. Strength and sensation are grossly intact. Psy',
 'ncer. She passed secondary to this. He does not know his father. He thinks his father passed from a cerebral hemorrage. His niece had breast cancer diagnosed in her 50s. He had one sister who passed from a fi',
 'a. Skin: No rashes or lesions. Neurological: Awake, alert and oriented x 3. Attention, language and memory are intact. Cranial nerve II-XII are grossly intact. Strength and sensation are grossly intact. Psy',
 'ncer. She passed secondary to this. He does not know his father. He thinks his father passed from a cerebral hemorrage. His niece had breast cancer di

In [138]:
df = pd.DataFrame(total_note_txt, columns = ["text"])
df = df.drop_duplicates(subset='text', keep="first")
df = df.replace(r'^\s*$', np.NaN, regex=True)
df.dropna()
total_note_txt = df["text"].to_list()

In [139]:
total_note_txt

['ncer. She passed secondary to this. He does not know his father. He thinks his father passed from a cerebral hemorrage. His niece had breast cancer diagnosed in her 50s. He had one sister who passed from a fi',
 'a. Skin: No rashes or lesions. Neurological: Awake, alert and oriented x 3. Attention, language and memory are intact. Cranial nerve II-XII are grossly intact. Strength and sensation are grossly intact. Psy',
 'cular: Normal rate. Pulmonary/Chest: Effort normal. No stridor. Genitourinary: Penis normal. Uncircumcised. No penile erythema. No discharge found. Musculoskeletal: Normal range of motion. He exhibits no',
 'ractures, obesity, insulin resistance, dyslipidemia, greater risk for diabetes, CVD and concern for dementia. While the safety of IAD is well established for men starting ADT in the absence of visible metasta',
 nan,
 'ions Thought Content: Appropriate to situation Sensorium: Alert, Oriented to person, place and time Memory: No deficits noted Fund of Knowledge: Ap

In [277]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_1.csv")

In [278]:
p1['regex_sent'] = p1['regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [279]:
p1['NoteTXT'] = p1['NoteTXT'].astype(str)
p1 = p1.drop_duplicates(subset='regex_sent', keep="first")
p1 = p1.replace(r'^\s*$', np.NaN, regex=True)
p1.reset_index(drop = True)
p1['regex_sent'] = p1.groupby(['PatientEncounterID'])['regex_sent'].transform(lambda x : ' ----- '.join(x))
p1 = p1.drop_duplicates(subset='PatientEncounterID', keep="first")

In [272]:
p1 = p1.replace(r'^\s*$', np.NaN, regex=True)
p1.reset_index(drop = True)
p1['regex_sent'] = p1.groupby(['PatientEncounterID'])['regex_sent'].transform(lambda x : ' ----- '.join(x))

In [273]:
p1["regex_location"] = p1["regex_location"].astype(str)

In [280]:
p1["regex_location"]

0                                        [(5850, 5858)]
1                                        [(3643, 3649)]
13                                       [(3947, 3950)]
14                                       [(8741, 8749)]
15    [(1048, 1052), (4878, 4887), (5104, 5110), (51...
20           [(5467, 5476), (5693, 5699), (5703, 5712)]
23                                       [(1984, 1990)]
24                                         [(859, 862)]
25    [(281, 290), (579, 588), (2148, 2154), (2163, ...
45                                         [(313, 319)]
46    [(7509, 7518), (7735, 7741), (7745, 7754), (78...
50                                       [(1473, 1476)]
Name: regex_location, dtype: object

In [275]:
p1 = p1.drop_duplicates(subset='PatientEncounterID', keep="first")
len(p1)

23

In [281]:
p1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\p1_updated_9_correct.csv", index = False)

In [81]:
x = ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862))
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(str(y))
z = len(p1[(p1["regex_location"] == str(y)) & (p1["PatientEncounterID"] == int(encounters[21]))]['NoteTXT'].values[1])
z

[(7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862)]


9187

In [99]:
x = [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)]
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(y)

[(181, 390), (479, 688), (2048, 2269), (6184, 6390), (6584, 6821), (7450, 7659), (7716, 7932), (9864, 10072), (11192, 11648), (12118, 12627)]


In [40]:
len(p1[(p1["regex_location"] == "[" +str(subset_loc[1]) + "]") & (p1["PatientEncounterID"] == int(encounters[1]))]['NoteTXT'].values[0])

5850

In [108]:
print(subset_loc_clone)

[[(5750, 5958)], [(3543, 3749)], [(5949, 6157)], [(3710, 3916)], [(5454, 5662)], [(5534, 5742)], [(5392, 5600)], [(5791, 5999)], [(5690, 5898)], [(6070, 6278)], [(6028, 6236)], [(5961, 6169)], [(6255, 6463)], [(3847, 4050)], [(8641, 8849)], [(948, 1152), (4778, 4987), (5004, 5210), (5014, 5223), (5127, 5331)], [(5367, 5576), (5593, 5799), (5603, 5812)], [(1884, 2090)], [(759, 962)], [(181, 390), (479, 688), (2048, 2254), (2063, 2269), (6184, 6390), (6584, 6793), (6618, 6821), (7450, 7659), (7716, 7920), (7728, 7932), (9864, 10072), (11192, 11401), (11283, 11503), (11414, 11623), (11444, 11648), (12118, 12338), (12283, 12492), (12387, 12607), (12409, 12612), (12419, 12627)], [(213, 419)], [(7409, 7618), (7635, 7841), (7645, 7854), (7758, 7962)], [(1373, 1576)]]
